<a href="https://colab.research.google.com/github/Gaurav7004/RSE_TOOL/blob/main/RSE_TOOL_29thMay2023_2_30_PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install pandarallel
# from pandarallel import pandarallel
import json
import re
import pandas as pd
import numpy as np
import itertools

In [2]:
# pandarallel.initialize()

In [3]:
df = pd.read_csv('Data1.csv')

# remove blank spaces from front and back of column names
df.columns = df.columns.str.strip()
df.columns

<ipython-input-3-fc8b3e05b5c4>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data1.csv')


Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST', 'GEDU',
       'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG'],
      dtype='object')

In [4]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example5.RSE', "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

{
    "#SAMPLING SUBSAMPLE2": [],
    "#GROUP 2": [
        "SEC   C sec",
        "AGE_GR C NEWVAR"
    ],
    "#VARIABLE 3  ": [
        "POP\tN  NEWVAR",
        "LF\tN  NEWVAR",
        "WRK\tN  NEWVAR"
    ],
    "#FILE Data1.Xlsx": [],
    "#RENAME  5    ": [
        "STRMID C sec+nss+strm+sstrm",
        "SS     C ss",
        "NSS    N  nss",
        "NSC    N  nsc",
        "MULT   N  mult"
    ],
    "#TRANSFORM 4  ": [
        "AGE_GR=(\"01\") AGE<15",
        "AGE_GR=(\"02\") AGE>=15 && AGE<20",
        "AGE_GR=(\"03\") AGE>=20 && AGE<30",
        "AGE_GR=(\"04\") AGE>=30 && AGE<60",
        "AGE_GR=(\"05\") Otherwise",
        "POP=(ERN_SELF+ERN_REG)",
        "WRK=(ERN_REG)  (NOT (CWS=\"11\" OR CWS=\"12\" OR CWS=\"21\")) AND GEDU in (\"5,6,7,11,12\")",
        "WRK=(0)  Otherwise",
        "LF=(ERN_SELF)   (NOT (CWS>=\"31\" AND CWS <\"41\")) AND GEDU>=\"05\"",
        "LF=(0)   Otherwise"
    ],
    "#EST_RSE 7": [
        "S POP",
        "E POP",
        "R POP",
      

In [5]:
def SAMPLING_(data, df):
    for key, value in data.items():
        ## To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')

            elif str('SRSWR').lower() in str(key).lower():
                pass
            elif str('SRSWOR').lower() in str(key).lower():
                pass

SAMPLING_(data, df)

'SUBSAMPLE2'

In [6]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST', 'GEDU',
       'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG'],
      dtype='object')

In [7]:
### ---------------------------------------------------------------------------------------------------------
def GROUP_(data, df):
    for key, value in data.items():    
        if str('GROUP').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            for name in value:
                if name in df.columns:
                    pass
                elif name not in df.columns:
                    df[name] = 0

            return value


    for key, value in data.items():
        print(key, value)
        ### To read the group columns
        if str('GROUP').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\w+\s+\w+\s+\w+'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst = match.group().split()

                print(lst[0], lst[1])

                if lst[0] in df.columns:
                    if lst[1] == 'C':
                        df[lst[0]] = df[lst[0]].astype(str)
                elif lst[0] not in df.columns:
                    # if lst[1] == 'C':
                    #     df[lst[0]] = ''
                    pass

        else:
            pass

### ---------------------------------------------------------------------------------------------------------
def FILTER_(data, df):
    lst_queries = []
    for key, value in data.items():

        if str('FILTER').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\([^()]+\)|\w+[<>!=]+[0-9]+(?:\s*(?:&&|\|\|)\s*(?:\([^()]+\)|\w+[<>!=]+[0-9]+))*'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst_queries.append(match.group())

        else:
            pass

    filter_query = ' & '.join(['({})'.format(item) for item in lst_queries])
   
    try:
        df = df.query(filter_query)
        return df
    except:
        return df


### ---------------------------------------------------------------------------------------------------------
def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)   
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

### ---------------------------------------------------------------------------------------------------------
def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if int(i) < 10 else str(i) for i in x]), axis=1)


                else:
                    col_name = col_name.upper()

### ---------------------------------------------------------------------------------------------------------
def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            
            for name in value:
                df[name] = 1

### ---------------------------------------------------------------------------------------------------------
def TRANSFORM_(data):
    global condition

    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                print(line_values)
                var_name = line_values[0].strip()  # extract the variable name
                
                var_value = line_values[1].strip().split()[0]  # extract the variable value

                            
                # Regular expression pattern to extract the condition
                pattern = r'^\w+=\([^\)]*\)(?:\s+(.*?)(?:\n|$))?$'

                # Split the string into lines and extract the condition from each line
                conditions = []

                for line in line.split('\n'):
                    match = re.match(pattern, line)
                 
                    if match:
                        condition = match.group(1)
                        
                        if condition is not None:
                            conditions.append(condition.strip())
                        else:
                            conditions.append(None)

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)

                Arr3.append(condition)

    return Arr1, Arr2, Arr3
        
### ---------------------------------------------------------------------------------------------------------
def EST_RSE_(data):
    for key, value in data.items():
        pass

df = FILTER_(data, df)
GROUP_(data, df)
TRANSFORM_(data)
RENAME_(data, df)
NEWVARIABLE_(data, df)
FILE_(data)



['AGE_GR', '("01") AGE<15']
['AGE_GR', '("02") AGE>', '15 && AGE<20']
['AGE_GR', '("03") AGE>', '20 && AGE<30']
['AGE_GR', '("04") AGE>', '30 && AGE<60']
['AGE_GR', '("05") Otherwise']
['POP', '(ERN_SELF+ERN_REG)']
['WRK', '(ERN_REG)  (NOT (CWS', '"11" OR CWS', '"12" OR CWS', '"21")) AND GEDU in ("5,6,7,11,12")']
['WRK', '(0)  Otherwise']
['LF', '(ERN_SELF)   (NOT (CWS>', '"31" AND CWS <"41")) AND GEDU>', '"05"']
['LF', '(0)   Otherwise']


In [8]:
GROUP_(data, df)

['SEC', 'AGE_GR']

In [9]:
EST_RSE_(data)

In [10]:
def parse_query_string(query_string):
    # Replace curly quotes with straight quotes
    query_string = query_string.replace("”", "\"")
    
    if 'in' in query_string:
        if ('NOT in' or 'not in') in query_string:
            query_string = re.sub(r'(\b\w+\b)\s+NOT in\s*\(([^)]+)\)', r'~\1.isin([\2])', query_string)
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        elif 'in' in query_string:
            # Replace 'in' operators with 'isin' and add list brackets
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        else:
            pass
            
    else:
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)('[^']*'|"[^"]*")|(\b\w+\b)(\s*[<>=!]+\s*)("[^]*'|"[^"]*")''', r'\1\2\3', query_string)
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)(\d+)''', r'\1\2\3', query_string)

    # Replace double quotes with single quotes
    query_string = query_string.replace("\"", "'")
    
    # Replace 'AND' with '&', 'OR' with '|', and add parentheses
    query_string = query_string.replace(" AND ", " & ")\
                        .replace(" OR ", " | ").\
                            replace("&&", '&').\
                                replace('||', '|').\
                                    replace('<>', '!=').\
                                        replace('NOT', '~')
    
    # Replace column names with df[column] syntax
    query_string = re.sub(r'(\b\w+\b)=', r'\1==', query_string)


    # # Define a function to replace variable names with df["<variable_name>"]
    # def replace_var(match):
    #     return f'df["{match.group(0)}"]'

    # # Use regex to replace variable names with df["<variable_name>"]
    # pattern = re.compile(r'''\b\w+\b(?=\s*[<>=!]=*\s*[\'"\d])''')
    # new_string = pattern.sub(replace_var, query_string)

    query_string = re.sub(r"'(\d+)'|\"(\d+)\"", lambda match: str(int(match.group(1) or match.group(2))), query_string)
    query_string = re.sub(r'\[(\d+,\s*)+\d+\]', lambda match: '[' + match.group(0).replace("\'", "'") + ']', query_string)
    
    return query_string


final_parsed_query = []

for i in list(TRANSFORM_(data))[2]:
    # print(i)
    if i:
        parsed_query = parse_query_string(i)
        final_parsed_query.append(parsed_query)
    else:
        final_parsed_query.append(i)

### Calling the final_parsed_query function
### ---------------------------------------
final_parsed_query

['AGE_GR', '("01") AGE<15']
['AGE_GR', '("02") AGE>', '15 && AGE<20']
['AGE_GR', '("03") AGE>', '20 && AGE<30']
['AGE_GR', '("04") AGE>', '30 && AGE<60']
['AGE_GR', '("05") Otherwise']
['POP', '(ERN_SELF+ERN_REG)']
['WRK', '(ERN_REG)  (NOT (CWS', '"11" OR CWS', '"12" OR CWS', '"21")) AND GEDU in ("5,6,7,11,12")']
['WRK', '(0)  Otherwise']
['LF', '(ERN_SELF)   (NOT (CWS>', '"31" AND CWS <"41")) AND GEDU>', '"05"']
['LF', '(0)   Otherwise']


['AGE<15',
 'AGE>=15 & AGE<20',
 'AGE>=20 & AGE<30',
 'AGE>=30 & AGE<60',
 'Otherwise',
 None,
 "(~ (CWS==11 | CWS==12 | CWS==21)) & GEDU.isin(['5,6,7,11,12'])",
 'Otherwise',
 '(~ (CWS>=31 & CWS <41)) & GEDU>=5',
 'Otherwise']

In [11]:
df['AGE'].unique()

array([ 40,  18,  19,  38,  21,  28,   1,  52,  26,  45,  25,  36,  17,
        30,  55,  44,  16,  20,  68,  35,  10,  42,  14,  39,  65,  43,
        15,  60,  48,  22,   8,   9,  13,  27,  24,  62,  23,   0,  56,
         2,   4,  66,  47,  50,  57,  61,  58,  31,   7,  34,  64,   6,
        54,  11,  29,  32,  41,  59,  51,  75,  70,  12,  37,  33,  85,
        72,   3,   5,  53,  73,  46,  67,  49,  80,  63,  78,  71,  82,
        81,  69,  74,  86,  76,  84,  83,  79,  95,  87,  90,  77,  89,
       110,  96, 100,  88,  93,  92,  98,  94, 105,  91,  99,  97, 109])

In [12]:
Arr1 = list(TRANSFORM_(data))[0]
Arr2 = list(TRANSFORM_(data))[1]

# print(Arr2)
Arr2 = [i.strip('()\'\"') for i in Arr2]
Arr3 = final_parsed_query

Arr2 = [float(elem) if elem.replace('.', '', 1).isdigit() else elem for elem in Arr2]
print(Arr2)

# print(Arr1)
# print(Arr2)
# print(Arr3)

['AGE_GR', '("01") AGE<15']
['AGE_GR', '("02") AGE>', '15 && AGE<20']
['AGE_GR', '("03") AGE>', '20 && AGE<30']
['AGE_GR', '("04") AGE>', '30 && AGE<60']
['AGE_GR', '("05") Otherwise']
['POP', '(ERN_SELF+ERN_REG)']
['WRK', '(ERN_REG)  (NOT (CWS', '"11" OR CWS', '"12" OR CWS', '"21")) AND GEDU in ("5,6,7,11,12")']
['WRK', '(0)  Otherwise']
['LF', '(ERN_SELF)   (NOT (CWS>', '"31" AND CWS <"41")) AND GEDU>', '"05"']
['LF', '(0)   Otherwise']
['AGE_GR', '("01") AGE<15']
['AGE_GR', '("02") AGE>', '15 && AGE<20']
['AGE_GR', '("03") AGE>', '20 && AGE<30']
['AGE_GR', '("04") AGE>', '30 && AGE<60']
['AGE_GR', '("05") Otherwise']
['POP', '(ERN_SELF+ERN_REG)']
['WRK', '(ERN_REG)  (NOT (CWS', '"11" OR CWS', '"12" OR CWS', '"21")) AND GEDU in ("5,6,7,11,12")']
['WRK', '(0)  Otherwise']
['LF', '(ERN_SELF)   (NOT (CWS>', '"31" AND CWS <"41")) AND GEDU>', '"05"']
['LF', '(0)   Otherwise']
[1.0, 2.0, 3.0, 4.0, 5.0, 'ERN_SELF+ERN_REG', 'ERN_REG', 0.0, 'ERN_SELF', 0.0]


In [13]:
df.head(3)

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,PAS,PS_SS,CWS,ERN_SELF,ERN_REG,AGE_GR,STRMID,POP,LF,WRK
0,1,28,17469,1,1,1,1,1,1,1,...,51,51,51,0,0,0,01010101,1,1,1
1,1,28,17469,1,1,1,1,1,1,1,...,91,91,91,0,0,0,01010101,1,1,1
2,1,28,17469,1,1,1,1,1,1,1,...,91,91,91,0,0,0,01010101,1,1,1


In [14]:
set(Arr1) - set(list(df.columns))

set()

In [15]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

set()

In [16]:
Arr3

['AGE<15',
 'AGE>=15 & AGE<20',
 'AGE>=20 & AGE<30',
 'AGE>=30 & AGE<60',
 'Otherwise',
 None,
 "(~ (CWS==11 | CWS==12 | CWS==21)) & GEDU.isin(['5,6,7,11,12'])",
 'Otherwise',
 '(~ (CWS>=31 & CWS <41)) & GEDU>=5',
 'Otherwise']

In [17]:
Arr3 = [i.replace("(['", "([").replace("'])", "])") if i is not None else 'True' if Arr3[j] == 'Otherwise' else True for j, i in enumerate(Arr3)]
Arr3

['AGE<15',
 'AGE>=15 & AGE<20',
 'AGE>=20 & AGE<30',
 'AGE>=30 & AGE<60',
 'Otherwise',
 True,
 '(~ (CWS==11 | CWS==12 | CWS==21)) & GEDU.isin([5,6,7,11,12])',
 'Otherwise',
 '(~ (CWS>=31 & CWS <41)) & GEDU>=5',
 'Otherwise']

In [18]:
new_arr3 = []
for i, value in enumerate(Arr3):
    if value == 'Otherwise':
        other_values = set(['~(' + Arr3[j] + ')' for j in range(len(Arr3)) if Arr1[j] == Arr1[i] and Arr3[j] != 'Otherwise'])
        new_value = ' & '.join(other_values)
        print(new_value)
        new_arr3.append(new_value)
    else:
        new_arr3.append(value)
        print(value)

# new_arr3

AGE<15
AGE>=15 & AGE<20
AGE>=20 & AGE<30
AGE>=30 & AGE<60
~(AGE>=15 & AGE<20) & ~(AGE>=30 & AGE<60) & ~(AGE>=20 & AGE<30) & ~(AGE<15)
True
(~ (CWS==11 | CWS==12 | CWS==21)) & GEDU.isin([5,6,7,11,12])
~((~ (CWS==11 | CWS==12 | CWS==21)) & GEDU.isin([5,6,7,11,12]))
(~ (CWS>=31 & CWS <41)) & GEDU>=5
~((~ (CWS>=31 & CWS <41)) & GEDU>=5)


In [19]:
Arr3 = new_arr3

In [20]:
df['WRK'].unique()

array([1])

In [21]:
Arr2

[1.0, 2.0, 3.0, 4.0, 5.0, 'ERN_SELF+ERN_REG', 'ERN_REG', 0.0, 'ERN_SELF', 0.0]

In [22]:
for i in range(len(Arr2)):
    if type(Arr3[i]) == str:
        print(Arr3[i])

AGE<15
AGE>=15 & AGE<20
AGE>=20 & AGE<30
AGE>=30 & AGE<60
~(AGE>=15 & AGE<20) & ~(AGE>=30 & AGE<60) & ~(AGE>=20 & AGE<30) & ~(AGE<15)
(~ (CWS==11 | CWS==12 | CWS==21)) & GEDU.isin([5,6,7,11,12])
~((~ (CWS==11 | CWS==12 | CWS==21)) & GEDU.isin([5,6,7,11,12]))
(~ (CWS>=31 & CWS <41)) & GEDU>=5
~((~ (CWS>=31 & CWS <41)) & GEDU>=5)


In [23]:
# print()
df.query("CWS==31 & GEDU.isin([5,6,7,11,12])")

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,PAS,PS_SS,CWS,ERN_SELF,ERN_REG,AGE_GR,STRMID,POP,LF,WRK
69,1,28,17471,4,1,2,2,1,2,1,...,31,31,31,0,7000,0,01010102,1,1,1
90,1,28,17471,8,1,3,2,1,2,1,...,31,31,31,0,4500,0,01010102,1,1,1
208,1,28,17476,3,1,2,2,1,4,2,...,31,31,31,0,8000,0,01010104,1,1,1
228,1,28,17477,1,1,1,1,1,5,1,...,31,31,31,0,40000,0,01010105,1,1,1
255,1,28,17478,1,1,1,1,1,5,2,...,31,31,31,0,6000,0,01010105,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109815,2,34,27427,7,1,4,1,1,3,2,...,31,31,31,0,7000,0,02030103,1,1,1
109819,2,34,27428,1,1,1,1,1,3,2,...,31,31,31,0,25000,0,02030103,1,1,1
109832,2,34,27428,4,1,2,2,1,3,2,...,31,31,31,0,15000,0,02030103,1,1,1
109847,2,34,27428,7,1,4,1,1,3,2,...,31,31,31,0,15000,0,02030103,1,1,1


In [24]:
# Apply the queries in Arr3 to filter the rows of the dataframe
for i, query_str in enumerate(Arr3):
    
    if query_str is True:
        if type(Arr2[i]) == str:
            if '+' in Arr2[i]:
                df[Arr1[i]] = df[Arr2[i].split('+')].apply(lambda x: x.sum(), axis=1)
                print('*** First if', Arr2[i], Arr1[i], Arr3[i], i)

            else:
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                print('*** First else', Arr2[i], Arr1[i], Arr3[i], i)
        # if type(Arr3[i]):
        #     print('^^^ Second if', Arr2[i], Arr1[i], Arr3[i], i)
        #     df.loc[df_query.index, Arr1[i]] = Arr2[i]
    
    else:
        # try:
        # query_str = re.sub(r'0?\s?(\d+)', r'\1', query_str)
        print(query_str)
        df_query = df.query(query_str)
        # except:
        #     pass

        if type(Arr2[i]) == float:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('### Third if', Arr2[i], Arr1[i], Arr3[i])
        else:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('@@@ First else', Arr2[i], Arr1[i], Arr3[i])


        if type(Arr2[i]) == str:
            if Arr2[i] in list(df.columns):
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                # print('$$$ Fourth if', Arr2[i], Arr1[i], Arr3[i])

# print(df)

AGE<15
AGE>=15 & AGE<20
AGE>=20 & AGE<30
AGE>=30 & AGE<60
~(AGE>=15 & AGE<20) & ~(AGE>=30 & AGE<60) & ~(AGE>=20 & AGE<30) & ~(AGE<15)
*** First if ERN_SELF+ERN_REG POP True 5
(~ (CWS==11 | CWS==12 | CWS==21)) & GEDU.isin([5,6,7,11,12])
~((~ (CWS==11 | CWS==12 | CWS==21)) & GEDU.isin([5,6,7,11,12]))
(~ (CWS>=31 & CWS <41)) & GEDU>=5
~((~ (CWS>=31 & CWS <41)) & GEDU>=5)


In [25]:
# # Apply the queries in Arr3 to filter the rows of the dataframe
# for i, query_str in enumerate(Arr3):
    
#     if query_str is True:
#         print(query_str)

#         try:
#             query_str = re.sub(r'(?<=\d)\s+(?=\d)', '', query_str)
#             df_query = df.query(query_str)
#         except:
#             pass

#         if type(Arr2[i]) == str:
#             if '+' in Arr2[i]:
#                 df[Arr1[i]] = df[Arr2[i].split('+')].apply(lambda x: x.sum(), axis=1)
#                 print('*** First if', Arr2[i], Arr1[i], Arr3[i], i)

#             if True:
#                 # try:
#                 query_str = re.sub(r'(?<=\d)\s+(?=\d)', '', query_str)
#                 df_query = df.query(query_str)
#                 # except:
#                 #     pass
#                 df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
#                 print('*** First else', Arr2[i], Arr1[i], Arr3[i], i)

#         else:
#             df.loc[df_query.index, Arr1[i]] = Arr2[i]
#             print('*** 2nd else', Arr2[i], Arr1[i], Arr3[i], i)
    
#     else:
#         query_str = re.sub(r'(?<=\d)\s+(?=\d)', '', query_str)
#         # try:
#         df_query = df.query(query_str)
#         # except:
#         #     pass

#         if type(Arr2[i]) == float:
#             df.loc[df_query.index, Arr1[i]] = Arr2[i]
#             # print('### 2nd if', Arr2[i], Arr1[i], Arr3[i])
#         else:
#             df.loc[df_query.index, Arr1[i]] = Arr2[i]
#             print('@@@ 3rd else', Arr2[i], Arr1[i], Arr3[i])


#         if type(Arr2[i]) == str:
#             if Arr2[i] in list(df.columns):
#                 df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
#                 print('$$$ 3rd if', Arr2[i], Arr1[i], Arr3[i])

# # print(df)

In [26]:
df[['LF']].value_counts()

LF   
0        98409
10000     1113
8000       882
15000      868
5000       828
         ...  
2290         1
10820        1
10900        1
10950        1
13400        1
Length: 445, dtype: int64

In [27]:
# ### Decide the type of the Variables
# ### --------------------------------
    
# def GROUP_Decide_Type(data):
#     for key, value in data.items():    
#         if str('GROUP').lower() in str(key).lower():
#             value_name = [s.split()[0] for s in value]
#             value_type = [s.split()[1] for s in value]
#             return {k:v for k, v in zip(value_name, value_type)}


# def VARIABLE_Decide_Type(data):
#     for key, value in data.items():    
#         if str('VARIABLE').lower() in str(key).lower():
#             value_name = [s.split()[0] for s in value]
#             value_type = [s.split()[1] for s in value]

#             return {k:v for k, v in zip(value_name, value_type)}


# def RENAME_Decide_Type(data):
#     for key, value in data.items():    
#         if str('RENAME').lower() in str(key).lower():
#             value_name = [s.split()[0] for s in value]
#             value_type = [s.split()[1] for s in value]

#             return {k:v for k, v in zip(value_name, value_type)}

# decide_group = GROUP_Decide_Type(data)
# decide_var = VARIABLE_Decide_Type(data)
# decide_rename = RENAME_Decide_Type(data)


# for k, v in decide_group.items():
#     if v == 'N':
#         print(v)
#         df[k] = pd.to_numeric(df[k])
    
#     if v == 'C' or 'c':
#         df[k] = df[k].astype(str)

# for i, j in decide_var.items():
#     if j == 'N' or 'n':
#         df[i] = pd.to_numeric(df[i])
    
#     if j == 'C' or 'c':
#         df[i] = df[i].astype(str)

# for m, n in decide_rename.items():
#     if n is 'N' or 'n':
#         df[m] = pd.to_numeric(df[m])
    
#     if n is 'C' or 'c':
#         df[m] = df[m].astype(str)

In [28]:
df.to_csv('WorkFile_Balanced_Half_Sample_Method_Example5.csv')

In [29]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST', 'GEDU',
       'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'AGE_GR', 'STRMID', 'POP',
       'LF', 'WRK'],
      dtype='object')

In [30]:
df['SEC'].unique()

array([1, 2])

**STEP 1**

In [61]:
# # Read the data from the CSV file
# df = pd.read_csv('Work_File1A.csv')

# Get the current column names
old_col_names = df.columns.tolist()

# Create a dictionary with the new column names
new_col_names = {col: col.strip() for col in old_col_names}

# Rename the columns using the dictionary
df.rename(columns=new_col_names, inplace=True)

df['MULT'] = pd.to_numeric(df['MULT'])

# Calculate the SS_MULT and CMULT fields
df['SS_MULT'] = df['MULT'] / 100
df['CMULT'] = df.apply(lambda row: row['MULT'] / 200 if row['NSC'] > row['NSS'] else row['MULT'] / 100, axis=1)

lst_df = list(df.columns)

groupby_cols = GROUP_(data, df)

# Create a list of all possible subsets of groupby_cols
groupby_cols_subsets = [list(subset) for i in range(len(groupby_cols) + 1)
                        for subset in itertools.combinations(groupby_cols, i)]

# Select columns that are not in groupby_cols
cols = [col for col in df.columns if col in groupby_cols]

agg_dict = {col: sum for col in cols if (df[col].dtype == int or df[col].dtype == float) and col in groupby_cols}

################
Arr_W1 = []


### Loop over each subset of columns and perform the aggregation
### ------------------------------------------------------------
for groupby_cols_subset in groupby_cols_subsets:
    # Define the prefix to use for the new columns
    prefix = 'Z'

    if len(groupby_cols_subset) == 0:

        # # If empty list is selected, perform aggregation without grouping
        # w1 = df.agg(agg_dict).to_frame().T

        # print(new_df)

        test_lst = groupby_cols

        df['POP'] = pd.to_numeric(df['POP']) * df['CMULT']
        df['LF'] = pd.to_numeric(df['LF']) * df['CMULT']
        df['WRK'] = pd.to_numeric(df['WRK']) * df['CMULT']
        df['MULT'] = df.shape[0]

        agg_dict = {'POP': 'sum', 'LF': 'sum', 'WRK': 'sum'}

        # Group the data by SEC and ST_GR and calculate the aggregates
        w1 = df.agg(agg_dict).to_frame().T

        # Rename the columns and reset the index
        w1 = w1.rename(columns={'POP': 'pophat', 'LF': 'lfhat', 'WRK': 'wrkhat', 'MULT': 'count'}).reset_index()
        w1['groupby_cols'] = 'All columns'

        new_col_val_ = []

        for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w1.columns)))):
            # Create a new column with the appropriate number of "Z"s and the prefix
            new_col_val_.append(prefix * (i+1))

        for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w1.columns)))):
            # Add the new column to the DataFrame
            w1[test_lst[i]] = new_col_val_[i]

        # print(w1)
        ################
        Arr_W1.append(w1)

        continue

    # Create a new dataframe by grouping by the selected columns and aggregating
    # Note: here I'm just calculating the sum of columns C and D, but you can use any aggregation function
    # new_df = df.groupby(list(groupby_cols_subset)).agg(agg_dict).reset_index()
    
    # Group the data by SEC and ST_GR and calculate the aggregates
    w1 = df.groupby(groupby_cols_subset).agg({'POP': lambda x: (pd.to_numeric(x) * df['CMULT']).sum(),
                                                'LF': lambda x: (pd.to_numeric(x) * df['CMULT']).sum(),
                                                    'WRK': lambda x: (pd.to_numeric(x) * df['CMULT']).sum(),
                                                        'MULT': 'count'}).reset_index()

    # Rename the columns and reset the index
    w1 = w1.rename(columns={'POP': 'pophat', 'LF': 'lfhat', 'WRK': 'wrkhat', 'MULT': 'no_sam'}).reset_index()

    test_lst = groupby_cols_subset

    # Add a new column to indicate which groupby columns are selected
    w1['groupby_cols'] = ', '.join(groupby_cols_subset)

    new_col_val = []
    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w1.columns)))):
        # Create a new column with the appropriate number of "Z"s and the prefix
        new_col_val.append(prefix * (i+1))

    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w1.columns)))):
        # Add the new column to the DataFrame
        w1[list(set(groupby_cols_subsets[-1]) - set(w1.columns))[i]] = new_col_val[i]

    Arr_W1.append(w1)


In [62]:
Arr_W1

[   index        pophat         lfhat        wrkhat groupby_cols SEC AGE_GR
 0      0  1.443412e+15  6.185486e+14  3.732672e+14  All columns   Z     ZZ,
    index  SEC        pophat         lfhat        wrkhat  no_sam groupby_cols  \
 0      0    1  3.839722e+17  1.930662e+17  6.886093e+16   62248          SEC   
 1      1    2  4.105608e+17  1.477973e+17  1.365466e+17   47608          SEC   
 
   AGE_GR  
 0      Z  
 1      Z  ,
    index  AGE_GR        pophat         lfhat        wrkhat  no_sam  \
 0      0       1  5.650106e+14  1.833875e+14  3.041124e+14   26055   
 1      1       2  7.296130e+15  1.625116e+15  2.964095e+15   12104   
 2      2       3  1.245872e+17  4.144304e+16  4.833664e+16   20108   
 3      3       4  6.022275e+17  2.642711e+17  1.494338e+17   41296   
 4      4       5  5.985707e+16  3.334086e+16  4.368904e+15   10293   
 
   groupby_cols SEC  
 0       AGE_GR   Z  
 1       AGE_GR   Z  
 2       AGE_GR   Z  
 3       AGE_GR   Z  
 4       AGE_GR   Z  ,
    

In [63]:
df['SS'].unique()

array([1, 2])

In [64]:
df['no_sam'] = 0

In [65]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST', 'GEDU',
       'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'AGE_GR', 'STRMID', 'POP',
       'LF', 'WRK', 'SS_MULT', 'CMULT', 'no_sam', 'popstr', 's1pop', 's2pop',
       'lfstr', 's1lf', 's2lf', 'wrkstr', 's1wrk', 's2wrk'],
      dtype='object')

**STEP 2** IT is running slow, figure it out later...


In [73]:
# Filter dataframe to include only rows where SS is 1 or 2
df = df.query("SS in [1, 2]")
# print(f"Number of rows after filtering: {len(df)}")

Arr_W2 = []

### Loop over subset of columns to perform aggregation
for i in range(len(groupby_cols_subsets)):

    new_groupby_cols_subset = ['STRMID']

    # Define the prefix to use for the new columns
    prefix = 'Z'

    if i == 0:

        df['popstr'] = pd.to_numeric(df['POP']) * df['CMULT']
        df['s1pop'] = np.where(df['SS'] == 1, pd.to_numeric(df['POP']) * df['SS_MULT'], 0)
        df['s2pop'] = np.where(df['SS'] == 2, pd.to_numeric(df['POP']) * df['SS_MULT'], 0)
        df['lfstr'] = pd.to_numeric(df['LF']) * df['CMULT']
        df['s1lf'] = np.where(df['SS'] == 1, pd.to_numeric(df['LF']) * df['SS_MULT'], 0)
        df['s2lf'] = np.where(df['SS'] == 2, pd.to_numeric(df['LF']) * df['SS_MULT'], 0)
        df['wrkstr'] = pd.to_numeric(df['WRK']) * df['CMULT']
        df['s1wrk'] = np.where(df['SS'] == 1, pd.to_numeric(df['WRK']) * df['SS_MULT'], 0)
        df['s2wrk'] = np.where(df['SS'] == 2, pd.to_numeric(df['WRK']) * df['SS_MULT'], 0)

        agg_dict = {'popstr': 'sum', 's1pop': 'sum', 's2pop': 'sum', 'lfstr': 'sum', 's1lf': 'sum', \
                    's2lf': 'sum', 'wrkstr': 'sum', 's1wrk': 'sum', 's2wrk': 'sum'}

        # # If empty list is selected, perform aggregation without grouping
        w2 = df.agg(agg_dict).to_frame().T

        w2['STRMID'] = 'ALL'

        new_col_val_ = []

        for i in range(len(list(set(groupby_cols) - set(w2.columns)))):
            # Create a new column with the appropriate number of "Z"s and the prefix
            new_col_val_.append(prefix * (i+1))

        for i in range(len(list(set(groupby_cols) - set(w2.columns)))):
            # Add the new column to the DataFrame
            w2[groupby_cols[i]] = new_col_val_[i]

        Arr_W2.append(w2)

        continue

    print(groupby_cols_subsets[i])

    new_groupby_cols_subset.extend(groupby_cols_subsets[i])

    print(new_groupby_cols_subset)

    w2 = df.groupby(groupby_cols_subsets[i]).agg(
            popstr=('POP', lambda x: np.sum(x * df['CMULT'])),
            s1pop=('POP', lambda x: np.nansum(np.where(df['SS'] == 1, x * df['SS_MULT'], 0))),
                s2pop=('POP', lambda x: np.nansum(np.where(df['SS'] == 2, x * df['SS_MULT'], 0))),
                lfstr=('LF', lambda x: np.nansum(x * df['CMULT'])),
                    s1lf=('LF', lambda x: np.nansum(np.where(df['SS'] == 1, x * df['SS_MULT'], 0))),
                    s2lf=('LF', lambda x: np.nansum(np.where(df['SS'] == 2, x * df['SS_MULT'], 0))),
                        wrkstr=('WRK', lambda x: np.nansum(x * df['CMULT'])),
                        s1wrk=('WRK', lambda x: np.nansum(np.where(df['SS'] == 1, x * df['SS_MULT'], 0))),
                        s2wrk=('WRK', lambda x: np.nansum(np.where(df['SS'] == 2, x * df['SS_MULT'], 0)))
                            ).reset_index()

    new_col_val = []
    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w2.columns)))):
        # Create a new column with the appropriate number of "Z"s and the prefix
        new_col_val.append(prefix * (i+1))

    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w2.columns)))):
        # Add the new column to the DataFrame
        w2[list(set(groupby_cols_subsets[-1]) - set(w2.columns))[i]] = new_col_val[i]

    Arr_W2.append(w2)

['SEC']
['STRMID', 'SEC']
['AGE_GR']
['STRMID', 'AGE_GR']
['SEC', 'AGE_GR']
['STRMID', 'SEC', 'AGE_GR']


In [74]:
for i in Arr_W2:
    print(i.columns)

for i in Arr_W1:
    print(i.columns)

Index(['popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf', 'wrkstr', 's1wrk',
       's2wrk', 'STRMID', 'SEC', 'AGE_GR'],
      dtype='object')
Index(['SEC', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf', 'wrkstr',
       's1wrk', 's2wrk', 'AGE_GR'],
      dtype='object')
Index(['AGE_GR', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf', 'wrkstr',
       's1wrk', 's2wrk', 'SEC'],
      dtype='object')
Index(['SEC', 'AGE_GR', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf',
       'wrkstr', 's1wrk', 's2wrk'],
      dtype='object')
Index(['index', 'pophat', 'lfhat', 'wrkhat', 'groupby_cols', 'SEC', 'AGE_GR'], dtype='object')
Index(['index', 'SEC', 'pophat', 'lfhat', 'wrkhat', 'no_sam', 'groupby_cols',
       'AGE_GR'],
      dtype='object')
Index(['index', 'AGE_GR', 'pophat', 'lfhat', 'wrkhat', 'no_sam',
       'groupby_cols', 'SEC'],
      dtype='object')
Index(['index', 'SEC', 'AGE_GR', 'pophat', 'lfhat', 'wrkhat', 'no_sam',
       'groupby_cols'],
      dtype='object')


In [75]:
Arr_W3 = []

### Loop over subset of columns to perform aggregation
for i in range(len(Arr_W2)):

    # Join w2 and w1 on SEC and ST_GR columns
    # w3 = Arr_W2[i].merge(Arr_W1[i], on=['SEC', 'AGE_GR'])
    w3 = pd.concat([Arr_W2[i], Arr_W1[i]], axis=1)

    w3['no_sam'] = w3.shape[0]
    
    # Add new columns to w3
    w3['r1hat'] = w3['lfhat'] / w3['pophat']
    w3['r2hat'] = w3['wrkhat'] / w3['pophat']

    # Equate s1pop and s2pop
    w3.loc[w3['popstr'] == w3['s1pop'], 's1pop'] = w3.loc[w3['popstr'] == w3['s1pop'], 's2pop']
    w3.loc[w3['popstr'] == w3['s2pop'], 's2pop'] = w3.loc[w3['popstr'] == w3['s2pop'], 's1pop']

    # Equate s1lf and s2lf
    w3.loc[w3['lfstr'] == w3['s1lf'], 's1lf'] = w3.loc[w3['lfstr'] == w3['s1lf'], 's2lf']
    w3.loc[w3['lfstr'] == w3['s2lf'], 's2lf'] = w3.loc[w3['lfstr'] == w3['s2lf'], 's1lf']

    # Equate s1wrk and s2wrk
    w3.loc[w3['wrkstr'] == w3['s1wrk'], 's1wrk'] = w3.loc[w3['wrkstr'] == w3['s1wrk'], 's2wrk']
    w3.loc[w3['wrkstr'] == w3['s2wrk'], 's2wrk'] = w3.loc[w3['wrkstr'] == w3['s2wrk'], 's1wrk']
    
    Arr_W3.append(w3)



In [76]:
Arr_W3

[         popstr         s1pop         s2pop         lfstr          s1lf  \
 0  7.945329e+17  7.993634e+17  7.863115e+17  3.408635e+17  3.475222e+17   
 
            s2lf        wrkstr         s1wrk         s2wrk STRMID  ... index  \
 0  3.319905e+17  2.054076e+17  2.092795e+17  2.007770e+17    ALL  ...     0   
 
          pophat         lfhat        wrkhat  groupby_cols  SEC AGE_GR no_sam  \
 0  1.443412e+15  6.185486e+14  3.732672e+14   All columns    Z     ZZ      1   
 
       r1hat     r2hat  
 0  0.428532  0.258601  
 
 [1 rows x 22 columns],
    SEC        popstr         s1pop         s2pop         lfstr          s1lf  \
 0    1  3.839722e+17  3.860585e+17  3.791775e+17  1.930662e+17  1.979511e+17   
 1    2  4.105608e+17  4.133049e+17  4.071340e+17  1.477973e+17  1.495711e+17   
 
            s2lf        wrkstr         s1wrk         s2wrk  ... index  SEC  \
 0  1.860575e+17  6.886093e+16  6.816423e+16  6.921668e+16  ...     0    1   
 1  1.459330e+17  1.365466e+17  1.411152e+1

In [77]:
Arr_W3[3].columns

Index(['SEC', 'AGE_GR', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf',
       'wrkstr', 's1wrk', 's2wrk', 'index', 'SEC', 'AGE_GR', 'pophat', 'lfhat',
       'wrkhat', 'no_sam', 'groupby_cols', 'r1hat', 'r2hat'],
      dtype='object')

In [78]:
### Drop duplicate columns
for i in range(len(Arr_W3)):

    Arr_W3[i] = Arr_W3[i].loc[:, ~Arr_W3[i].columns.duplicated()]

    print(Arr_W3[i].columns)

Index(['popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf', 'wrkstr', 's1wrk',
       's2wrk', 'STRMID', 'SEC', 'AGE_GR', 'index', 'pophat', 'lfhat',
       'wrkhat', 'groupby_cols', 'no_sam', 'r1hat', 'r2hat'],
      dtype='object')
Index(['SEC', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf', 'wrkstr',
       's1wrk', 's2wrk', 'AGE_GR', 'index', 'pophat', 'lfhat', 'wrkhat',
       'no_sam', 'groupby_cols', 'r1hat', 'r2hat'],
      dtype='object')
Index(['AGE_GR', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf', 'wrkstr',
       's1wrk', 's2wrk', 'SEC', 'index', 'pophat', 'lfhat', 'wrkhat', 'no_sam',
       'groupby_cols', 'r1hat', 'r2hat'],
      dtype='object')
Index(['SEC', 'AGE_GR', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf',
       'wrkstr', 's1wrk', 's2wrk', 'index', 'pophat', 'lfhat', 'wrkhat',
       'no_sam', 'groupby_cols', 'r1hat', 'r2hat'],
      dtype='object')


In [79]:
Arr_W3[3]['SEC'].value_counts()

1    5
2    5
Name: SEC, dtype: int64

In [80]:
agg_dict

{'popstr': 'sum',
 's1pop': 'sum',
 's2pop': 'sum',
 'lfstr': 'sum',
 's1lf': 'sum',
 's2lf': 'sum',
 'wrkstr': 'sum',
 's1wrk': 'sum',
 's2wrk': 'sum'}

In [81]:
Arr_W4 = []


### Loop over subset of columns to perform aggregation
### Loop over subset of columns to perform aggregation
for i in range(len(groupby_cols_subsets)):

    if i == 0:
        gby_list = ['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat']

        w4 = Arr_W3[i].groupby(gby_list).agg(

        var_pop = ('s1pop', lambda x: np.sum((x - Arr_W3[i]['s2pop']) ** 2)),

                        mse_R1 = ('s1lf', lambda x: np.nansum(
                                    ((x - Arr_W3[i]['s2lf']) ** 2) +
                                        ((Arr_W3[i]['r1hat'] ** 2) * ((Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop']) ** 2) -
                                            (2 * Arr_W3[i]['r1hat'] * (x - Arr_W3[i]['s2lf']) * (Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop'])))
                                )),

                            mse_R2 = ('s1wrk', lambda x: np.nansum(
                                        ((x - Arr_W3[i]['s2wrk']) ** 2) +
                                            ((Arr_W3[i]['r2hat'] ** 2) * ((Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop']) ** 2) -
                                                (2 * Arr_W3[i]['r2hat'] * (x - Arr_W3[i]['s2wrk']) * (Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop'])))
                                    ),
                    
                                )).reset_index()

        Arr_W4.append(w4)

        continue

    gby_list = ['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat']

    gby_list.extend(groupby_cols_subsets[i])

    print(gby_list)

    w4 = Arr_W3[i].groupby(gby_list).agg(

    var_pop = ('s1pop', lambda x: np.sum((x - Arr_W3[i]['s2pop']) ** 2)),

                    mse_R1 = ('s1lf', lambda x: np.nansum(
                                ((x - Arr_W3[i]['s2lf']) ** 2) +
                                    ((Arr_W3[i]['r1hat'] ** 2) * ((Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop']) ** 2) -
                                        (2 * Arr_W3[i]['r1hat'] * (x - Arr_W3[i]['s2lf']) * (Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop'])))
                            )),

                        mse_R2 = ('s1wrk', lambda x: np.nansum(
                                    ((x - Arr_W3[i]['s2wrk']) ** 2) +
                                        ((Arr_W3[i]['r2hat'] ** 2) * ((Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop']) ** 2) -
                                            (2 * Arr_W3[i]['r2hat'] * (x - Arr_W3[i]['s2wrk']) * (Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop'])))
                                ),
                
                            )).reset_index()

    Arr_W4.append(w4)

['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC']
['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'AGE_GR']
['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC', 'AGE_GR']


In [82]:
Arr_W4

[   no_sam        pophat         lfhat        wrkhat     r1hat     r2hat  \
 0       1  1.443412e+15  6.185486e+14  3.732672e+14  0.428532  0.258601   
 
         var_pop        mse_R1        mse_R2  
 0  1.703539e+32  9.877509e+31  2.628858e+31  ,
    no_sam        pophat         lfhat        wrkhat     r1hat     r2hat  SEC  \
 0       2  3.839722e+17  1.930662e+17  6.886093e+16  0.502813  0.179338    1   
 1       2  4.105608e+17  1.477973e+17  1.365466e+17  0.359989  0.332586    2   
 
         var_pop        mse_R1        mse_R2  
 0  4.734825e+31  7.112841e+31  5.228009e+30  
 1  3.808078e+31  2.006903e+30  5.628852e+31  ,
    no_sam        pophat         lfhat        wrkhat     r1hat     r2hat  \
 0       5  5.650106e+14  1.833875e+14  3.041124e+14  0.324574  0.538242   
 1       5  7.296130e+15  1.625116e+15  2.964095e+15  0.222737  0.406256   
 2       5  5.985707e+16  3.334086e+16  4.368904e+15  0.557008  0.072989   
 3       5  1.245872e+17  4.144304e+16  4.833664e+16  0.3326

In [83]:
Arr_W4 = []

### Loop over subset of columns to perform aggregation
for i in range(len(Arr_W3)):
    # mask = Arr_W3[i].columns.duplicated()
    # Arr_W3[i] = Arr_W3[i].loc[:, ~mask]

    gby_list = ['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat'] + groupby_cols

    print(gby_list)

    w4 = Arr_W3[i].groupby(gby_list).agg(

        var_pop = ('s1pop', lambda x: np.sum((x - Arr_W3[i]['s2pop']) ** 2)),

            mse_R1 = ('s1lf', lambda x: np.nansum(
                        ((x - Arr_W3[i]['s2lf']) ** 2) +
                            ((Arr_W3[i]['r1hat'] ** 2) * ((Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop']) ** 2) -
                                (2 * Arr_W3[i]['r1hat'] * (x - Arr_W3[i]['s2lf']) * (Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop'])))
                    )),

                mse_R2 = ('s1wrk', lambda x: np.nansum(
                            ((x - Arr_W3[i]['s2wrk']) ** 2) +
                                ((Arr_W3[i]['r2hat'] ** 2) * ((Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop']) ** 2) -
                                    (2 * Arr_W3[i]['r2hat'] * (x - Arr_W3[i]['s2wrk']) * (Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop'])))
                        ),
        
    )).reset_index()

    Arr_W4.append(w4)

    print(Arr_W4[i])

['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC', 'AGE_GR']
   no_sam        pophat         lfhat        wrkhat     r1hat     r2hat SEC  \
0       1  1.443412e+15  6.185486e+14  3.732672e+14  0.428532  0.258601   Z   

  AGE_GR       var_pop        mse_R1        mse_R2  
0     ZZ  1.703539e+32  9.877509e+31  2.628858e+31  
['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC', 'AGE_GR']
   no_sam        pophat         lfhat        wrkhat     r1hat     r2hat  SEC  \
0       2  3.839722e+17  1.930662e+17  6.886093e+16  0.502813  0.179338    1   
1       2  4.105608e+17  1.477973e+17  1.365466e+17  0.359989  0.332586    2   

  AGE_GR       var_pop        mse_R1        mse_R2  
0      Z  4.734825e+31  7.112841e+31  5.228009e+30  
1      Z  3.808078e+31  2.006903e+30  5.628852e+31  
['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC', 'AGE_GR']
   no_sam        pophat         lfhat        wrkhat     r1hat     r2hat SEC  \
0       5  5.650106e+14  1.8338

In [84]:
Arr_W5 = []
for i in range(len(Arr_W4)):
    # Calculate RSEs for each variable in T1
    w5 = Arr_W4[i].assign(
        RSE_POP=lambda x: 100 * (x['var_pop'] ** 0.5) / (2 * x['pophat']),
        RSE_R1=lambda x: 100 * (x['mse_R1'] ** 0.5) / (2 * x['lfhat']),
        RSE_R2=lambda x: 100 * (x['mse_R2'] ** 0.5) / (2 * x['wrkhat'])
    )

    Arr_W5.append(w5)

In [85]:
Arr_W5

[   no_sam        pophat         lfhat        wrkhat     r1hat     r2hat SEC  \
 0       1  1.443412e+15  6.185486e+14  3.732672e+14  0.428532  0.258601   Z   
 
   AGE_GR       var_pop        mse_R1        mse_R2     RSE_POP      RSE_R1  \
 0     ZZ  1.703539e+32  9.877509e+31  2.628858e+31  452.122041  803.377947   
 
        RSE_R2  
 0  686.805465  ,
    no_sam        pophat         lfhat        wrkhat     r1hat     r2hat  SEC  \
 0       2  3.839722e+17  1.930662e+17  6.886093e+16  0.502813  0.179338    1   
 1       2  4.105608e+17  1.477973e+17  1.365466e+17  0.359989  0.332586    2   
 
   AGE_GR       var_pop        mse_R1        mse_R2   RSE_POP    RSE_R1  \
 0      Z  4.734825e+31  7.112841e+31  5.228009e+30  0.896029  2.184165   
 1      Z  3.808078e+31  2.006903e+30  5.628852e+31  0.751529  0.479255   
 
      RSE_R2  
 0  1.660219  
 1  2.747255  ,
    no_sam        pophat         lfhat        wrkhat     r1hat     r2hat SEC  \
 0       5  5.650106e+14  1.833875e+14  3.041

In [86]:
Tables = []

for i in range(len(Arr_W5)):
    gby_list = ['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'RSE_POP', 'RSE_R1', 'RSE_R2'] + groupby_cols
    # assuming the data is loaded into a DataFrame called w5
    T1 = Arr_W5[i][gby_list].copy()

    # add empty columns for sec_desc and st_gr_desc
    T1['sec_desc'] = ''
    T1['st_gr_desc'] = ''

    # apply formatting to columns
    T1['R1'] = 100 * T1['r1hat']
    T1['R2'] = 100 * T1['r2hat']

    gby_list = ['sec_desc', 'st_gr_desc', 'no_sam', 'pophat','lfhat', 'wrkhat', 'RSE_POP', 'R1', 'RSE_R1', 'R2', 'RSE_R2'] + groupby_cols

    # reorder columns
    T1 = T1[gby_list]

    gby_list = ['no_sam', 'pophat','lfhat', 'wrkhat', 'r1hat', 'r2hat', 'RSE_POP', 'RSE_R1', 'RSE_R2'] + groupby_cols

    T1 = Arr_W5[i][gby_list].copy()

    T1['sec_desc'] = ' '
    T1['st_gr_desc'] = ' '

    T1['R1'] = 100 * T1['r1hat']
    T1['R2'] = 100 * T1['r2hat']

    gby_list = ['no_sam', 'pophat','lfhat', 'wrkhat', 'RSE_POP', 'R1', 'RSE_R1', 'R2', 'RSE_R2'] + groupby_cols

    T1 = T1[gby_list]

    Tables.append(T1)
    

print(Tables)
print('\n')

[   no_sam        pophat         lfhat        wrkhat     RSE_POP        R1  \
0       1  1.443412e+15  6.185486e+14  3.732672e+14  452.122041  42.85322   

       RSE_R1         R2      RSE_R2 SEC AGE_GR  
0  803.377947  25.860057  686.805465   Z     ZZ  ,    no_sam        pophat         lfhat        wrkhat   RSE_POP         R1  \
0       2  3.839722e+17  1.930662e+17  6.886093e+16  0.896029  50.281288   
1       2  4.105608e+17  1.477973e+17  1.365466e+17  0.751529  35.998895   

     RSE_R1         R2    RSE_R2  SEC AGE_GR  
0  2.184165  17.933833  1.660219    1      Z  
1  0.479255  33.258570  2.747255    2      Z  ,    no_sam        pophat         lfhat        wrkhat    RSE_POP         R1  \
0       5  5.650106e+14  1.833875e+14  3.041124e+14  43.819404  32.457361   
1       5  7.296130e+15  1.625116e+15  2.964095e+15  10.528522  22.273673   
2       5  5.985707e+16  3.334086e+16  4.368904e+15   7.625645  55.700800   
3       5  1.245872e+17  4.144304e+16  4.833664e+16   2.337844  

In [87]:
def merge_dataframes(tables):
    merged_df = tables[0]
    for df in tables[1:]:
        merged_df = pd.concat([merged_df, df], axis=0)
    return merged_df

merge_dataframes(Tables)

,no_sam,pophat,lfhat,wrkhat,RSE_POP,R1,RSE_R1,R2,RSE_R2,SEC,AGE_GR
0,1,1.443412e+15,6.185486e+14,3.732672e+14,452.122041,42.853220,803.377947,25.860057,686.805465,Z,ZZ
0,2,3.839722e+17,1.930662e+17,6.886093e+16,0.896029,50.281288,2.184165,17.933833,1.660219,1,Z
1,2,4.105608e+17,1.477973e+17,1.365466e+17,0.751529,35.998895,0.479255,33.258570,2.747255,2,Z
0,5,5.650106e+14,1.833875e+14,3.041124e+14,43.819404,32.457361,22.426637,53.824195,27.135747,Z,1
1,5,7.296130e+15,1.625116e+15,2.964095e+15,10.528522,22.273673,8.011070,40.625574,1.011573,Z,2
2,5,5.985707e+16,3.334086e+16,4.368904e+15,7.625645,55.700800,5.902861,7.298894,26.382473,Z,5
3,5,1.245872e+17,4.144304e+16,4.833664e+16,2.337844,33.264283,1.570497,38.797431,2.751655,Z,3
4,5,6.022275e+17,2.642711e+17,1.494338e+17,0.978021,43.882266,0.435591,24.813515,2.504134,Z,4
0,10,1.611962e+14,2.383146e+13,6.092927e+13,83.764903,14.784133,2.411609,37.798212,19.423582,2,1
1,10,4.038145e+14,1.595561e+14,2.431832e+14,27.873809,39.512228,36.115798,60.221507,23.130728,1,1


In [88]:
merge_dataframes(Tables).to_csv('Example1__Table(T).csv')

In [ ]:
# T1.to_csv('T1_Balanced_Half_Sample_Method.csv')

In [ ]:
T1.columns